# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
# import libraries
import matplotlib as plt
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from pathlib import Path

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('df_table', engine)  
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    ''' This function will tokenize out text message data into a machine readable format. It takes in text data'''
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    temp_text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens_pre = word_tokenize(temp_text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens_pre if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [6]:
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
print(classification_report(Y_test, Y_pred, target_names=Y.keys()))

                        precision    recall  f1-score   support

               related       0.69      0.82      0.75      1707
               request       0.46      0.18      0.26       899
                 offer       0.00      0.00      0.00         2
           aid_related       0.46      0.24      0.32       976
          medical_help       0.17      0.01      0.01       149
      medical_products       0.00      0.00      0.00        83
     search_and_rescue       0.00      0.00      0.00        55
              security       0.00      0.00      0.00        27
              military       0.00      0.00      0.00        16
           child_alone       0.00      0.00      0.00         0
                 water       0.07      0.01      0.02       180
                  food       0.25      0.03      0.06       373
               shelter       0.12      0.02      0.03       278
              clothing       0.00      0.00      0.00        22
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters =  parameters = {  
        'clf__estimator__min_samples_split': [2, 4],
    }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)
cv.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
accuracy = (Y_pred == Y_test).mean()
recall = recall_score(Y_test, Y_pred, average='macro')
pre = precision_score(Y_test, Y_pred, average='macro')

print("Recall:", recall)
print("Precision:", pre)
print("Accuracy:", accuracy)
print("\nBest Parameters:", cv.best_params_)

Recall: 0.0448258778853
Precision: 0.0921159594593
Accuracy: related                   0.630282
request                   0.638106
offer                     0.999218
aid_related               0.601721
medical_help              0.940141
medical_products          0.964397
search_and_rescue         0.978091
security                  0.988654
military                  0.993740
child_alone               1.000000
water                     0.919405
food                      0.843505
shelter                   0.878326
clothing                  0.990610
money                     0.988654
missing_people            0.991002
refugees                  0.982394
death                     0.974961
other_aid                 0.841941
infrastructure_related    0.971440
transport                 0.978482
buildings                 0.965962
electricity               0.992958
tools                     0.998435
hospitals                 0.996088
shops                     0.997261
aid_centers               0.9

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [40]:
path = open('classifier', 'wb')
pickle.dump(cv, path)
path.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.